## Part 1: Preprocessing

In [16]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [17]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [18]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [19]:
y_df.dtypes

Attrition     object
Department    object
dtype: object

In [20]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 
                    'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 
                    'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [21]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)

In [22]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

X_train['OverTime'].value_counts()

OverTime
No     799
Yes    303
Name: count, dtype: int64

In [25]:
X_train.head()

,Education,Age,DistanceFromHome,JobSatisfaction,OverTime,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked
591,3,33,16,1,No,0,3,3,0,5
267,2,25,5,1,No,2,3,6,1,1
1236,5,36,13,1,Yes,3,3,2,2,5
788,3,28,10,3,No,0,4,8,1,3
1224,4,26,17,3,No,3,4,3,0,1


In [23]:
# Encode the OverTime column as it contains Yes/No values using OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
ohe_overtime = OneHotEncoder(sparse_output=False)

X_train_encoded = ohe_overtime.fit_transform(X_train[['OverTime']])
X_test_encoded = ohe_overtime.transform(X_test[['OverTime']])



In [26]:
# Get encoded column names
overtime_columns = ohe_overtime.get_feature_names_out(['OverTime'])

# Create a DataFrame with the encoded column names
overtime_encoded_train_df = pd.DataFrame(X_train_encoded, columns=overtime_columns, index=X_train.index)
overtime_encoded_test_df = pd.DataFrame(X_test_encoded, columns=overtime_columns, index=X_test.index)

# Drop the original OverTime column from the X_train and X_test DataFrames and concatenate the encoded columns
X_train_encoded_df = pd.concat([X_train.drop(columns=['OverTime']), overtime_encoded_train_df], axis=1)
X_test_encoded_df = pd.concat([X_test.drop(columns=['OverTime']), overtime_encoded_test_df], axis=1)

In [27]:
X_train_encoded_df.head()

,Education,Age,DistanceFromHome,JobSatisfaction,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked,OverTime_No,OverTime_Yes
591,3,33,16,1,0,3,3,0,5,1.0,0.0
267,2,25,5,1,2,3,6,1,1,1.0,0.0
1236,5,36,13,1,3,3,2,2,5,0.0,1.0
788,3,28,10,3,0,4,8,1,3,1.0,0.0
1224,4,26,17,3,3,4,3,0,1,1.0,0.0


In [28]:
# Create a StandardScaler
std_scaler = StandardScaler()

# Fit the StandardScaler to the training data ignore index column
X_scaler = std_scaler.fit(X_train_encoded_df)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train_encoded_df)
X_test_scaled = X_scaler.transform(X_test_encoded_df)


In [29]:
# Convert scaled data to a DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_encoded_df.columns, index=X_train_encoded_df.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_encoded_df.columns, index=X_test_encoded_df.index)

In [39]:
X_train_scaled_df.head()

,Education,Age,DistanceFromHome,JobSatisfaction,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked,OverTime_No,OverTime_Yes
591,0.105287,-0.409424,0.874754,-1.541729,-0.925250,0.34094,-0.638148,-0.674305,0.920043,0.615812,-0.615812
267,-0.886389,-1.298897,-0.507205,-1.541729,1.413341,0.34094,-0.136312,-0.345069,-0.673493,0.615812,-0.615812
1236,2.088639,-0.075872,0.497856,-1.541729,2.582636,0.34094,-0.805427,-0.015834,0.920043,-1.623873,1.623873
788,0.105287,-0.965344,0.120958,0.261865,-0.925250,1.70718,0.198245,-0.345069,0.123275,0.615812,-0.615812
1224,1.096963,-1.187713,1.000387,0.261865,2.582636,1.70718,-0.638148,-0.674305,-0.673493,0.615812,-0.615812


In [34]:
# Create a OneHotEncoder for the Department column
ohe_department = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
ohe_department.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_department = ohe_department.transform(y_train[['Department']])
y_test_encoded_department = ohe_department.transform(y_test[['Department']])

# Get the encoded column names
department_columns = ohe_department.get_feature_names_out(['Department'])

# Create DataFrames with the encoded column names
department_encoded_train_df = pd.DataFrame(y_train_encoded_department, columns=department_columns, index=y_train.index)
department_encoded_test_df = pd.DataFrame(y_test_encoded_department, columns=department_columns, index=y_test.index)

In [35]:
# Create a OneHotEncoder for the Attrition column
ohe_attrition = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
ohe_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_attrition = ohe_attrition.transform(y_train[['Attrition']])
y_test_encoded_attrition = ohe_attrition.transform(y_test[['Attrition']])

# Get the encoded column names
attrition_columns = ohe_attrition.get_feature_names_out(['Attrition'])

# Create DataFrames with the encoded column names
attrition_encoded_train_df = pd.DataFrame(y_train_encoded_attrition, columns=attrition_columns, index=y_train.index)
attrition_encoded_test_df = pd.DataFrame(y_test_encoded_attrition, columns=attrition_columns, index=y_test.index)

In [38]:
X_train_scaled[:5]

array([[ 0.10528682, -0.40942386,  0.87475435, -1.54172867, -0.92525011,
         0.34094003, -0.63814846, -0.67430453,  0.92004345,  0.61581168,
        -0.61581168],
       [-0.88638904, -1.29889668, -0.50720476, -1.54172867,  1.41334076,
         0.34094003, -0.1363124 , -0.34506944, -0.67349349,  0.61581168,
        -0.61581168],
       [ 2.08863855, -0.07587155,  0.49785641, -1.54172867,  2.5826362 ,
         0.34094003, -0.80542715, -0.01583436,  0.92004345, -1.62387305,
         1.62387305],
       [ 0.10528682, -0.96534437,  0.12095847,  0.26186474, -0.92525011,
         1.70717974,  0.19824498, -0.34506944,  0.12327498,  0.61581168,
        -0.61581168],
       [ 1.09696269, -1.18771258,  1.000387  ,  0.26186474,  2.5826362 ,
         1.70717974, -0.63814846, -0.67430453, -0.67349349,  0.61581168,
        -0.61581168]])

## Create, Compile, and Train the Model

In [49]:
# Find the number of columns in the X training data
num_features = X_train_scaled.shape[1]
print(f"Number of input features: {num_features}")

# Create the input layer
input_layer = layers.Input(shape=(num_features,))

# Create at least two shared layers
shared_layer_1 = layers.Dense(units=128, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(units=64, activation='relu')(shared_layer_1)

Number of input features: 11


In [50]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch = layers.Dense(units=32, activation='relu')(shared_layer_2)

# Create the output layer
department_output = layers.Dense(units=department_encoded_train_df.shape[1], activation='softmax', name='department_output')(department_branch)


In [51]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = layers.Dense(units=32, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output = layers.Dense(units=attrition_encoded_train_df.shape[1], activation='sigmoid', name='attrition_output')(attrition_branch)


In [52]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
                loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
                metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │      1,536 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │      8,256 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │      2,080 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_8[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         33 │ dense_9[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,084 (55.02 KB)

 Trainable params: 14,084 (55.02 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
# Train the model
model.fit(X_train_scaled, 
            {'department_output': department_encoded_train_df, 'attrition_output': attrition_encoded_train_df},
            epochs=100,
            batch_size=32,
            shuffle=True,
            verbose=2)


Epoch 1/100
35/35 - 1s - 23ms/step - attrition_output_accuracy: 0.8258 - attrition_output_loss: 0.4857 - department_output_accuracy: 0.6234 - department_output_loss: 0.8526 - loss: 1.3447
Epoch 2/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8276 - attrition_output_loss: 0.4170 - department_output_accuracy: 0.6661 - department_output_loss: 0.7620 - loss: 1.1782
Epoch 3/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8412 - attrition_output_loss: 0.3961 - department_output_accuracy: 0.6652 - department_output_loss: 0.7441 - loss: 1.1420
Epoch 4/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8548 - attrition_output_loss: 0.3774 - department_output_accuracy: 0.6661 - department_output_loss: 0.7381 - loss: 1.1168
Epoch 5/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0.8584 - attrition_output_loss: 0.3693 - department_output_accuracy: 0.6679 - department_output_loss: 0.7330 - loss: 1.0996
Epoch 6/100
35/35 - 0s - 2ms/step - attrition_output_accuracy: 0

In [54]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(X_test_scaled), {
    'department_output': department_encoded_test_df,
    'attrition_output': attrition_encoded_test_df
})

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8541 - attrition_output_loss: 0.9939 - department_output_accuracy: 0.5266 - department_output_loss: 3.2441 - loss: 4.2445  


In [55]:
print(results)

[4.1571526527404785, 3.0563647747039795, 1.058653712272644, 0.845108687877655, 0.5380434989929199]


In [56]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {results[3]*100:.2f}%")
print(f"Attrition Accuracy: {results[4]*100:.2f}%")

Department Accuracy: 84.51%
Attrition Accuracy: 53.80%


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric because the dataset is imbalanced. For example, if Attrition has significantly more “No” values than “Yes”, accuracy may give a misleading sense of performance as predicting “No” for all instances could yield high accuracy but poor performance for “Yes”.
2. Softmax and Sigmoid
    1. I have chosen *Softmax* activation for `Department` output because it is a multi-class classification task since we have 3 departments and there may be unknown departments for prediction and unknown data.
    2. I have chosen *Sigmoid* activation for `Attrition` output as it is a binary classification task with possible Yes or No answer to all known and unknown data
3. We can possibly do Regularization, Hyperparameter Tuning and Feature Engineering (especially for unbalanced data). We can also adjust the depth of layers and see how model works.